In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'QuadroP2000'

In [3]:
!cd ../../../prediction_model/data/raw_data/QuadroP2000/VGG16_imsize32 && pwd -P

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/VGG16_imsize32


In [10]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data/{}/VGG16_imsize32".format(model_name))
SAVE_DIR = os.path.abspath("../../../prediction_model/data/VGG16_32_100thnd")
print("Saving to {}".format(SAVE_DIR))

Saving to /HDD2/ML/ml-performance-prediction/prediction_model/data/VGG16_32_100thnd


## Dense Layer

In [43]:
filename = os.path.join(DATA_DIR,'benchmark_dense_{}_20190922.pkl'.format(model_name,model_name))
print(filename)
dfDense = pd.read_pickle(filename)

# for i in range(3,4):
#     dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20180925.pkl' %(file_name, i)))])

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/VGG16_imsize32/benchmark_dense_QuadroP2000_20190922.pkl


In [44]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [45]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [46]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,memory_weights,memory_in,memory_out,opt_None,act_None,act_relu,act_sigmoid,act_tanh
count,10000.000000,10000.00000,10000.000000,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,10000.000000,10000.000000,10000.000000,10000.000000
mean,247.762200,2043.69430,2047.076900,0.0,32.0,2.550096,2.533467,2.516837,0.016629,1.034045e+09,4.176571e+06,5.077407e+05,5.062380e+05,1.0,0.246100,0.251700,0.254400,0.247800
std,142.891785,1182.25642,1182.789959,0.0,0.0,1.559952,1.521055,1.507200,0.197339,1.209248e+09,3.703029e+06,4.476858e+05,4.465660e+05,0.0,0.430759,0.434011,0.435545,0.431757
min,1.000000,1.00000,1.000000,0.0,32.0,0.970364,0.964681,0.954707,0.000000,1.616000e+04,5.600000e+02,2.200000e+01,5.000000e+00,1.0,0.000000,0.000000,0.000000,0.000000
25%,125.000000,1015.75000,1021.000000,0.0,32.0,1.430035,1.417637,1.402696,0.005126,1.670992e+08,1.132352e+06,1.375748e+05,1.369440e+05,1.0,0.000000,0.000000,0.000000,0.000000
50%,247.000000,2034.00000,2051.000000,0.0,32.0,1.994650,1.980027,1.968503,0.010967,5.674764e+08,3.090940e+06,3.736695e+05,3.779955e+05,1.0,0.000000,0.000000,0.000000,0.000000
75%,371.000000,3071.00000,3059.000000,0.0,32.0,3.187974,3.172914,3.159384,0.019828,1.465518e+09,6.408668e+06,7.813865e+05,7.722510e+05,1.0,0.000000,1.000000,1.000000,0.000000
max,500.000000,4094.00000,4095.000000,0.0,32.0,41.404645,21.712820,9.696325,19.691825,7.650013e+09,1.658118e+07,2.021942e+06,2.016329e+06,1.0,1.000000,1.000000,1.000000,1.000000


In [47]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [5]:
!ls /HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/VGG16_imsize32

benchmark_convolution_QuadroP2000_20190922.csv
benchmark_convolution_QuadroP2000_20190922.npy
benchmark_convolution_QuadroP2000_20190922.pkl
benchmark_convolution_QuadroP2000_20190923.csv
benchmark_convolution_QuadroP2000_20190923.npy
benchmark_convolution_QuadroP2000_20190923.pkl
benchmark_dense_QuadroP2000_20190922.csv
benchmark_dense_QuadroP2000_20190922.npy
benchmark_dense_QuadroP2000_20190922.pkl


In [6]:
file_name = os.path.join(DATA_DIR,'benchmark_convolution_{}_20190923.pkl'.format(model_name,model_name))
print(file_name)
dfConv = pd.read_pickle(file_name)

header = dfConv.columns

# for i in range(2,3):
#     dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20180925.pkl' %(file_name, i)))])

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/VGG16_imsize32/benchmark_convolution_QuadroP2000_20190923.pkl


In [7]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})
# dfConv['optimizer'] = "opt_SGD"

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [8]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [9]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,elements_kernel,ops,memory_weights,memory_in,memory_out,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,100000.00000,100000.00000,100000.000000,100000.0,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,...,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,100000.0,100000.000000,100000.000000,100000.000000,100000.000000
mean,250.09650,247.03683,248.566710,0.0,3.665890,16.523390,0.499830,32.0,2.498490,37.721419,...,17.497130,2.893068e+08,9.880526e+05,1.227696e+06,3.631370e+05,1.0,0.249470,0.249140,0.251200,0.250190
std,144.46749,1014.37637,1018.882277,0.0,2.014552,9.251543,0.500002,0.0,1.118012,167.385628,...,15.978836,1.295279e+09,5.827807e+06,1.059547e+06,6.334119e+05,0.0,0.432708,0.432517,0.433705,0.433125
min,1.00000,1.00000,1.000000,0.0,1.000000,1.000000,0.000000,32.0,1.000000,2.686024,...,1.000000,4.410000e+02,1.000000e+00,7.200000e+01,1.333333e+00,1.0,0.000000,0.000000,0.000000,0.000000
25%,125.00000,7.00000,7.000000,0.0,2.000000,8.000000,0.000000,32.0,1.000000,4.962206,...,4.000000,4.665591e+06,4.520000e+02,3.546022e+05,3.869986e+04,1.0,0.000000,0.000000,0.000000,0.000000
50%,250.00000,16.00000,16.000000,0.0,3.000000,17.000000,0.000000,32.0,2.000000,7.738531,...,9.000000,2.495183e+07,3.003500e+03,9.286485e+05,1.202040e+05,1.0,0.000000,0.000000,0.000000,0.000000
75%,375.00000,61.00000,62.000000,0.0,5.000000,25.000000,1.000000,32.0,3.000000,19.808292,...,25.000000,1.240600e+08,4.090750e+04,1.862080e+06,3.530685e+05,1.0,0.000000,0.000000,1.000000,1.000000
max,500.00000,9998.00000,9999.000000,0.0,7.000000,32.000000,1.000000,32.0,4.000000,4985.604048,...,49.000000,4.078952e+10,9.702058e+07,4.952112e+06,4.980519e+06,1.0,1.000000,1.000000,1.000000,1.000000


In [11]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))

In [12]:
print("Saved to",os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))

Saved to /HDD2/ML/ml-performance-prediction/prediction_model/data/VGG16_32_100thnd/Data_convolution_QuadroP2000.pkl
